In [9]:
import json
from tqdm.notebook import tqdm

import numpy as np
import re
import requests
import string

import json
import numpy as np
from sklearn.metrics import jaccard_score

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
#Load different json datasets so we can combine them
def load_json_data(file_path):
    # open the JSON file
    with open(file_path, 'r') as f:
        data = json.load(f)

    # create an empty list to store the text
    dirty_text = []

    # add the text to the list
    for text in data['text']:
        dirty_text.append(text)
    return dirty_text


In [7]:
text1 = load_json_data('/content/drive/MyDrive/Spot the Bot/Wiki-Files/333k_unclean_articles.json')
text2 = load_json_data('/content/drive/MyDrive/Spot the Bot/Wiki-Files/162500_unclean_articles.json')
text3 = load_json_data('/content/drive/MyDrive/Spot the Bot/Wiki-Files/73k_wikipedia_articles.json')
text4 = load_json_data('/content/drive/MyDrive/Spot the Bot/Wiki-Files/27.5K_unclean_articles.json')

In [6]:
print(f'Num of text1: {len(text1)}')
print(f'Num of text2: {len(text2)}')
print(f'Num of text3: {len(text3)}')
print(f'Num of text4: {len(text4)}')


Num of text1: 333000
Num of text2: 162500
Num of text3: 73137
Num of text4: 27500


In [6]:
#Clean the text from english texts, newline charecters and other non needed charecters 
class TextCleaner():
    """
    Persian text cleaner
    """
    def __init__(self):        
        self.min_ascii = '0600'
        self.max_ascii = '06FF'
        self.punctuation = string.punctuation + '،' + '؛' + '؟' + '؛' + '۔' + '»' + '«' + '-'
        self.one_space_regex = r"\s((\s)(\s+)?)?"
        self.text = None

        self.dict_punct = dict(zip(list(self.punctuation), np.repeat(' ', len(self.punctuation))))
        
    def remove_punct(self, text):
        table = str.maketrans(self.dict_punct)
        text = text.translate(table)
        return text
    
    def remove_num(self, text):
        num_pattern = r'[\u06F0-\u06F9]'
        text = re.sub(num_pattern, ' ', text)
        return text
    
    def remove_spaces(self, text):
        try:
            text = re.sub(self.one_space_regex, ' ', text)    
            text = text if text[0] != ' ' else text[1:]
            text = text if text[-1] != ' ' else text[:-1]
            return text
        
        except IndexError as e:
            return ''
    
    def is_fa_token(self, token):
        for ch in set(token):
            if ord(ch) < int(self.min_ascii, 16) or ord(ch) > int(self.max_ascii, 16):
                return False
        return True

    def remove_foreign_lang(self, text):
        clean_text = ''
        for token in text.split():
            if self.is_fa_token(token):
                clean_text += ' ' + token
        return clean_text[1:]
    
    def clean_text(self, text):
        text = self.remove_punct(text)
        text = self.remove_num(text)
        text = self.remove_spaces(text)
        text = self.remove_foreign_lang(text)
        text = self.remove_spaces(text)
        return text


In [8]:
cleaner = TextCleaner()

In [9]:
cleaned_text1 = [cleaner.clean_text(text) for text in text1]
cleaned_text2 = [cleaner.clean_text(text) for text in text2]
cleaned_text3 = [cleaner.clean_text(text) for text in text3]
cleaned_text4 = [cleaner.clean_text(text) for text in text4]

In [11]:
combined_dataset = cleaned_text1 + cleaned_text2 + cleaned_text3 + cleaned_text4


In [12]:
#Save the clean combined dataset
with open('/content/drive/MyDrive/Spot the Bot/Wiki-Files/combined_datasets.json', 'w') as f:
    json.dump(combined_dataset, f)

In [13]:
len(combined_dataset)

596137

In [10]:
with open("/content/drive/MyDrive/Spot the Bot/Wiki-Files/combined_datasets.json", 'r') as f:
        clean_corp = json.load(f)

In [11]:
len(clean_corp)

596137

In [12]:
clean_corp[1:10]

['مانوئل فیشراطلاعات شخصیزادروز سپتامبر سال زادگاه آلنقد ٫ متر فوت اینچ پست جوانان باشگاه فوتبال اشتوتگارت باشگاه فوتبال اولم باشگاه فوتبال گل باشگاه فوتبال اشتوتگارت قرضی قرضی باشگاه فوتبال هایدنهایم قرضی باشگاه فوتبال بایرن مونیخ تیم ملی تیم ملی فوتبال جوانان آلمان تیم ملی فوتبال جوانان آلمان تعداد و فقط مربوط به لیگ داخلی است مانوئل فیشر انگلیسی زادهٔ سپتامبر بازیکن فوتبال اهل آلمان است از که در آن بازی به باشگاه فوتبال بایرن مونیخ باشگاه فوتبال اشتوتگارت و باشگاه فوتبال هایدنهایم اشاره کرد وی همچنین در ملی فوتبال جوانان آلمان و جوانان آلمان بازی منابع ویرایش به آلمانی در دانشنامهٔ انگلیسی در آوریل این یک مقالهٔ خرد یک بازیکن فوتبال است با گسترش آن به کمک کنید',
 'یرای پاتنیواطلاعات شخصینام کامل مهٔ سال زادگاه الچهقد ٫ متر فوت اینچ پست هافبکاطلاعات باشگاهیباشگاه کنونی باشگاه فوتبال لینکولن رد جوانان باشگاه فوتبال گل باشگاه فوتبال الچه باشگاه فوتبال لینکولن رد ایمپس تعداد و فقط مربوط به لیگ داخلی است و آمار به روز شده در تاریخ مه یرای پاتنیو انگلیسی زادهٔ مهٔ بازیکن فوتبال اهل اسپانی

In [13]:
#Now we need to remove stop words
stop_words = []
with open('/content/drive/MyDrive/Spot the Bot/Persian-stopwords.txt') as f:
    stop_words = f.read().splitlines()

for i, text in enumerate(clean_corp):
    words = text.split()
    for word in words:
        if word in stop_words:
            words.remove(word)
    clean_corp[i] = ' '.join(words)


In [20]:
# !pip install PersianStemmer
# !pip install https://github.com/htaghizadeh/PersianStemmer-Python/archive/master.zip --upgrade

In [14]:
from PersianStemmer import PersianStemmer 
stemmer = PersianStemmer()
for i, text in enumerate(clean_corp):
    words = text.split()
    stemmed_words = [stemmer.stem(word) for word in words]
    clean_corp[i] = ' '.join(stemmed_words)


In [15]:
clean_corp[1:10]

['مانوئل فیشراطلاعات شخصیزادروز سپتامبر سال زادگاه آلنقد ٫ متر فوت اینچ پست جوان باشگاه فوتبال اشتوتگارت باشگاه فوتبال اول باشگاه فوتبال گل باشگاه فوتبال اشتوتگارت قرضی قرضی باشگاه فوتبال هایدنهایم قرضی باشگاه فوتبال بایرن مونیخ تیم ملی تیم ملی فوتبال جوان آلمان تیم ملی فوتبال جوان آلمان تعداد فقط مربوط لیگ داخلی مانوئل فیشر انگلیسی زادهٔ سپتامبر بازیکن فوتبال اهل آلمان از بازی باشگاه فوتبال بایرن مونیخ باشگاه فوتبال اشتوتگارت باشگاه فوتبال هایدنهایم اشاره وی ملی فوتبال جوان آلمان جوان آلمان بازی منبع ویرایش آلمانی در دانشنامهٔ انگلیسی در آوریل مقالهٔ خرد یک بازیکن فوتبال با گسترش به کمک',
 'یرای پاتنیواطلاعات شخصینام مهٔ سال زادگاه الچهقد ٫ متر فوت اینچ پست هافبکاطلاعات باشگاهیباشگاه باشگاه فوتبال لینکولن رد جوان باشگاه فوتبال گل باشگاه فوتبال الچه باشگاه فوتبال لینکولن رد ایمپس تعداد فقط مربوط لیگ داخلی آمار روز تاریخ مه یرای پاتنیو انگلیسی زادهٔ مهٔ بازیکن فوتبال اهل اسپانیا از بازی باشگاه فوتبال الچه و باشگاه فوتبال لینکولن رد ایمپس اشاره منبع ویرایش در دانشنامهٔ انگلیسی در مارس مق

In [16]:
with open('./content/drive/MyDrive/Spot the Bot/Wiki-Files/stemmed_no_stopwords_corpus.txt', 'w', encoding='utf-8') as f:
    for _, text in tqdm(clean_corp):
        f.write(text)
        f.write('\n')

In [18]:
with open('/content/drive/MyDrive/Spot the Bot/Wiki-Files/stemmed_no_stopwords_corpus.json', 'w') as f:
    json.dump(clean_corp, f)